In [1]:
# the aim of this project is to detect main characteristics for given picture and to return some pictures similar
# to reach our task, we use convolution neural network already trained by large picture corpus (ImageNet). 
# the highest layers learned to detect complex patern
# we flated the highest layer activation matrix and calculate the difference of array between the array associated to the 
# given picture and other pictures in our bank of images 

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import seaborn as sns
import os, sys
import pandas as pd 

from sklearn.utils import shuffle

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
from os import listdir, makedirs
from os.path import join, exists, expanduser

from keras.applications.xception import preprocess_input as preprocess_input_Xception

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import cv2

Using TensorFlow backend.


# Rotation d'images (brouillon)

Ce notebook implémente deux procéssus. Celui de génération d'images d'entrainement utilisées dans l'algorithme pix2pix pour le reconstitution d'images, celui de génération d'images d'entrainement utilisées en entrée de l'algorithme de détection d'angle. (voir notebooks dédiés à cet effet)

In [ ]:
"""crée des images d'entrainement découpées (en forme de losange). Ces images serviront pour l'entrainement
de l'algorithme pix2pix. Ces images représentent la seconde étape du projet rotation/reconstruction. C'est à dire 
qu'elles correspondent à la rotation d'images inclinées. La quantité de découpage dépend donc de l'angle de rotation
censé être l'opposé de l'angle d'inclinaison du paysage contenu dans l'image. Cet angle alpha est choisi aléatoirement"""

dim = 256

j_te = 0

# Open a file
path = "./images_reshaped_dim224"
dirs = os.listdir( path )

# itérer > équivaut à effectuer de l'augmentation de données (x4)
for i in range (4):

    for file in (dirs):
        
        if (str(file)[-4] == '.') or (str(file)[-3] == '.') or (str(file)[-5] == '.'):

            file_picture =  "./images_reshaped_dim224/" + str(file)
            #print (file_picture)   

            img_open = load_img(file_picture, target_size=(dim, dim))  # this is a PIL image  


            j_te = j_te + 1


            img_s = img_to_array(img_open)

            l = img_s.shape[0]
            
            # angle d'inclinaison
            alpha_deg = np.random.rand(1)*20 + 5
            print (alpha_deg)
            alpha = alpha_deg*np.pi/180

            x = l/(1 + np.absolute(np.sin(alpha[0])))
            a = x * np.absolute(np.sin(alpha[0]))
            b = x * np.absolute(np.cos(alpha[0]))
            d = a
            c = l - (a+b)   


            pts = np.array([[l,int(np.around(b+c))],[int(np.around(a+c)),l],[int(np.around(c)),int(np.around(l-b))],
                            [int(np.around(l-d)),int(np.around(c))]])

            ## (1) Crop the bounding rect
            rect = cv2.boundingRect(pts)
            x,y,w,h = rect
            croped = img_s[y:y+h, x:x+w].copy()

            ## (2) make mask
            pts = pts - pts.min(axis=0)

            mask = np.zeros(croped.shape[:2], np.uint8)
            cv2.drawContours(mask, [pts], -1, (255, 255, 255), -1, cv2.LINE_AA)

            ## (3) do bit-op
            dst = cv2.bitwise_and(croped, croped, mask=mask)

            ## (4) add the white background
            bg = np.ones_like(croped, np.uint8)*255
            cv2.bitwise_not(bg,bg, mask=mask)
            dst2 = bg+ dst

            mat = np.ones((l,l,3))*255
            mat[int(np.around(c))::,0:-int(np.around(c)),:] = dst2[:,:,:]
            
            # lets stack pictures together (useful for pix2pix implementation inputs)
            img_end = np.zeros((img_s.shape[0],img_s.shape[1]*2,3))

            img_end[:,:img_end.shape[1]//2,:] = img_s[:,:,:]
            img_end[:,img_end.shape[1]//2:,:] = mat[:,:,:]

            img = array_to_img(img_end)

            if (str(file)[-3:] == 'jpg' ) or (str(file)[-3:] == 'gif' )  or (str(file)[-3:] == 'JPG' ):

                img.save("./images_translat2/" + 'l{}1'.format(i) + str(file)[:-3] + "png" ) 

            else :

                img.save("./images_translat2/" + 'l{}1'.format(i) + str(file)[:-4] + "png" ) 

            pts = np.array([[int(np.around(b+c)),l],[l,int(np.around(a+c))],[int(np.around(l-b)),int(np.around(c))],
                            [int(np.around(c)),int(np.around(l-d))]])


            ## (1) Crop the bounding rect
            rect = cv2.boundingRect(pts)
            x,y,w,h = rect
            croped = img_s[y:y+h, x:x+w].copy()

            ## (2) make mask
            pts = pts - pts.min(axis=0)

            mask = np.zeros(croped.shape[:2], np.uint8)
            cv2.drawContours(mask, [pts], -1, (255, 255, 255), -1, cv2.LINE_AA)

            ## (3) do bit-op
            dst = cv2.bitwise_and(croped, croped, mask=mask)

            ## (4) add the white background
            bg = np.ones_like(croped, np.uint8)*255
            cv2.bitwise_not(bg,bg, mask=mask)
            dst2 = bg+ dst

            mat2 = np.ones((l,l,3))*255
            mat2[int(np.around(c))::,int(np.around(c))::,:] = dst2[:,:,:]
            
            # lets stack pictures together (useful for pix2pix implementation inputs)
            img_end = np.zeros((img_s.shape[0],img_s.shape[1]*2,3))

            img_end[:,:img_end.shape[1]//2,:] = img_s[:,:,:]
            img_end[:,img_end.shape[1]//2:,:] = mat2[:,:,:]

            img = array_to_img(img_end)

            if (str(file)[-3:] == 'jpg' ) or (str(file)[-3:] == 'gif' ) or (str(file)[-3:] == 'JPG' ):

                img.save("./images_translat2/" + 'r{}1'.format(i) + str(file)[:-3] + "png") 

            else :

                img.save("./images_translat2/" + 'r{}1'.format(i) + str(file)[:-4] + "png") 
            
            


[7.60914341]
[7.32821779]
[11.47396516]
[13.79838372]
[7.59930814]
[22.80455123]
[15.12387936]
[16.10768593]
[18.5543338]
[5.9701252]
[19.34491067]
[14.5263847]
[17.46290774]
[15.61530105]
[22.84402785]
[18.3308438]
[10.16038336]
[13.02172965]
[21.77883152]
[24.37332696]
[6.26594553]
[5.51666152]
[22.77912533]
[18.29177494]
[23.80862579]
[17.85785578]
[17.08339297]
[10.87910859]
[6.42518503]
[12.47918893]
[14.86583511]
[20.41927242]
[7.72845623]
[11.49285257]
[9.27161828]
[23.74924467]
[19.79918625]
[8.51369601]
[15.55012608]
[8.41285614]
[18.39257746]
[17.51604997]
[6.29476831]
[17.48813611]
[5.27603427]
[21.94690424]
[19.33373843]
[5.51982103]
[17.24714199]
[20.370068]
[23.72525431]
[10.57775862]
[16.40429076]
[9.04976731]
[11.88211908]
[19.05379063]
[19.06605557]
[12.46021858]
[10.32072475]
[20.7764683]
[16.09875341]
[10.89255914]
[10.29391741]
[12.98324886]
[10.20490125]
[10.22453332]
[21.22498009]
[22.95295503]
[24.79619455]
[20.76618632]
[22.16821371]
[20.53352651]
[24.34980357]


In [2]:
def rotateAndScale(img, scaleFactor = 0.5, degreesCCW = 30):
    '''fonction de rotation'''
    (oldY,oldX,oldz) = img.shape #note: numpy uses (y,x) convention but most OpenCV functions use (x,y)
    M = cv2.getRotationMatrix2D(center=(oldX/2,oldY/2), angle=degreesCCW, scale=scaleFactor) #rotate about center of image.

    #choose a new image size.
    newX,newY = oldX*scaleFactor,oldY*scaleFactor
    #include this if you want to prevent corners being cut off
    r = np.deg2rad(degreesCCW)
    newX,newY = (abs(np.sin(r)*newY) + abs(np.cos(r)*newX),abs(np.sin(r)*newX) + abs(np.cos(r)*newY))

    #the warpAffine function call, below, basically works like this:
    # 1. apply the M transformation on each pixel of the original image
    # 2. save everything that falls within the upper-left "dsize" portion of the resulting image.

    #So I will find the translation that moves the result to the center of that region.
    (tx,ty) = ((newX-oldX)/2,(newY-oldY)/2)
    M[0,2] += tx #third column of matrix holds translation, which takes effect after rotation.
    M[1,2] += ty

    rotatedImg = cv2.warpAffine(img, M, dsize=(int(newX),int(newY)))
    return rotatedImg

"""on applique une fonction de rotation sur les images originales (représentant un contenu jugé non incliné) 
de telle sorte d'obtenir des losanges au sein desquels on vient découper une image. Cette image carré de dimension plus 
faible représente donc un contenu (paysage) incliné d'un angle alpha aléatoirement. Ces images serviront d'entrainement 
pour la détection automatique d'angles
"""
dim = 256

j_te = 0

# Open a file
#path = "./images_reshaped_dim224"
path = "./images_translat"
dirs = os.listdir( path )

liste_name = []
liste_angle = []

for i in range (1):

    for file in (dirs):

        #file_picture =  "./images_reshaped_dim224/" + str(file)
        file_picture =  "./images_translat/" + str(file)
        print (file_picture)   
        if (str(file)[-3:] == 'jpg') or (str(file)[-3:] == 'png') or (str(file)[-3:] == 'gif') or (str(file)[-3:] == 'JPG'):
            
            img_open = load_img(file_picture, target_size=(dim, dim))  # this is a PIL image  
            
            j_te = j_te + 1


            img_s = img_to_array(img_open)

            l = img_s.shape[0]

            alpha_deg = np.random.rand(1)*15+10
            print (alpha_deg)
            #alpha = alpha_deg*np.pi/180
            
            mat = rotateAndScale(img=img_s, scaleFactor = 1, degreesCCW = -alpha_deg)
            
            picture = mat.copy()

            d = int(np.around(picture.shape[0]/2))
            for x in range (int(np.around(picture.shape[0]/2))):
                d = d - 1

                if (picture[d,d,0] == 0) and (picture[d,d,1] == 0) and (picture[d,d,2] == 0):
                    dist = np.sqrt((int(np.around(picture.shape[1]/2))-d)**2+(int(np.around(picture.shape[1]/2))-d)**2)
                    dist = int(np.around(dist))
                    print (dist)

                    break


            picture_new = np.zeros((int(np.around(picture.shape[1]))-2*d,int(np.around(picture.shape[1]))-2*d,3))
            picture_new[:,:,:] = picture[d:int(np.around(picture.shape[1]))-d,
                                        d:int(np.around(picture.shape[1]))-d,:]


            img = array_to_img(picture_new) 
            
            



            if (str(file)[-3:] == 'jpg' ) or (str(file)[-3:] == 'gif' )  or (str(file)[-3:] == 'JPG' ):

                img.save('./images_translat3/' + 'l{}1'.format(i) + str(file)[:-3] + "png" ) 
                liste_name.append( 'l{}1'.format(i) + str(file)[:-3] + "png")
                liste_angle.append(alpha_deg)

            else :

                img.save( './images_translat3/' + 'l{}1'.format(i) + str(file)[:-4] + "png" ) 
                liste_name.append( 'l{}1'.format(i) + str(file)[:-3] + "png")
                liste_angle.append(alpha_deg)
                
                
            mat = rotateAndScale(img=img_s, scaleFactor = 1, degreesCCW = alpha_deg)
            
            picture = mat.copy()

            d = int(np.around(picture.shape[0]/2))
            for x in range (int(np.around(picture.shape[0]/2))):
                d = d - 1

                if (picture[d,d,0] == 0) and (picture[d,d,0] == 0) and (picture[d,d,0] == 0):
                    dist = np.sqrt((int(np.around(picture.shape[1]/2))-d)**2+(int(np.around(picture.shape[1]/2))-d)**2)
                    dist = int(np.around(dist))
                    print (dist)

                    break


            picture_new = np.zeros((int(np.around(picture.shape[1]))-2*d,int(np.around(picture.shape[1]))-2*d,3))
            picture_new[:,:,:] = picture[d:int(np.around(picture.shape[1]))-d,
                                        d:int(np.around(picture.shape[1]))-d,:]


            img = array_to_img(picture_new)        



            if (str(file)[-3:] == 'jpg' ) or (str(file)[-3:] == 'gif' )  or (str(file)[-3:] == 'JPG' ):

                img.save('./images_translat/' + 'r{}1'.format(i) + str(file)[:-3] + "png" ) 
                liste_name.append( 'r{}1'.format(i) + str(file)[:-3] + "png")
                liste_angle.append(alpha_deg)

            else :

                img.save( './images_translat/' + 'r{}1'.format(i) + str(file)[:-4] + "png" ) 
                liste_name.append( 'r{}1'.format(i) + str(file)[:-3] + "png")
                liste_angle.append(alpha_deg)
                

d = {'name': liste_name, 'angle': liste_angle}
df = pd.DataFrame(data=d)
            
df.to_csv('liste_label.csv')
            
            
            

./images_reshaped_dim224/00-aurores-boreales-islande.jpg
[20.51958998]
141
141
./images_reshaped_dim224/000bb5b4bd72bc5232fdbf9f6a52f75a.jpg
[13.4451136]
151
151
./images_reshaped_dim224/0029a35ac175019c26c7d17f89e637c4.jpg
[21.82499466]
140
140
./images_reshaped_dim224/002c5a5054952d026a7e9d629b72a551.jpg
[21.87217344]
140
140
./images_reshaped_dim224/00729b4a2f632036841379a7c0e4bcb8.jpg
[19.00740289]
143
143
./images_reshaped_dim224/007930949819179a044133ccf8b953ba.jpg
[23.17724572]
140
140
./images_reshaped_dim224/009e4b8f5590e084f30e9898261ae41f.jpg
[22.62039118]
140
140
./images_reshaped_dim224/009ea8fa97e0824f65aae54a4e072f42.jpg
[24.02504154]
139
139
./images_reshaped_dim224/00ced4e254184a21c7b3a49f2c01c4a6.jpg
[19.87879093]
143
143
./images_reshaped_dim224/00f82c5e5df76d12ffa28e0666b024d0.jpg
[16.78515082]
147
147
./images_reshaped_dim224/00fb7acb539cc330beb45688060b7bdf.jpg
[20.98540459]
141
141
./images_reshaped_dim224/010ec93dced10ffc15d9035a12f08013.jpg
[19.5079724]
143
143

150
./images_reshaped_dim224/08fe178c108b9f21b4a75f32c3c8d57c.jpg
[24.45869773]
139
139
./images_reshaped_dim224/090c9c70bcd6c226eeba5c978d823b1c.jpg
[20.68878962]
141
141
./images_reshaped_dim224/0917822b72ed3e5038851bf49065978c.jpg
[18.03907248]
144
144
./images_reshaped_dim224/092aa4709a2f578b2a8bfad173b6f017.jpg
[24.36977785]
139
139
./images_reshaped_dim224/09326edb830ad9220edaf814becc8e74.jpg
[15.07801228]
148
148
./images_reshaped_dim224/0960983be7bee45459aeb0bebdc6d881.jpg
[24.23308557]
139
139
./images_reshaped_dim224/09609fcd42607128e497e2b269a9dd49.jpg
[15.46156528]
148
148
./images_reshaped_dim224/09709168a18eb08a316976ba6a265303.jpg
[15.63449372]
148
148
./images_reshaped_dim224/097a8bff96d76075e60a3a4317b5e1f3.jpg
[18.73404405]
144
144
./images_reshaped_dim224/09b543d5e30c2f2602a691d905f01bed.jpg
[15.75955797]
148
148
./images_reshaped_dim224/09bed5c4eba1e21892b576f2d737daa6.jpg
[11.69899839]
154
154
./images_reshaped_dim224/09ceac800d738af6da75dff1e126a0d5.jpg
[12.969995

./images_reshaped_dim224/0fc6a43f2de7c887ea556bbfb206b79e.jpg
[18.97238447]
143
143
./images_reshaped_dim224/0fcb97d3d6f653c90cb0e0ca2c55ce78.jpg
[21.4374008]
141
141
./images_reshaped_dim224/0ff860ae4901d3be959ab4db12b99372.jpg
[22.85675861]
140
140
./images_reshaped_dim224/0ffee75cee2170ede318cf66e2bd4df0.jpg
[10.22999035]
157
157
./images_reshaped_dim224/0_caters_stunning_milkyway_over_city_01.jpg
[24.56169615]
139
139
./images_reshaped_dim224/100ffd51594408b9b6e18bec4e5c3aba.jpg
[13.49708925]
151
151
./images_reshaped_dim224/101a7fc7e2ed6040d9a7687e2efbfd85.jpg
[10.80651836]
157
157
./images_reshaped_dim224/10218f47d0f4eb1c70959ad98845f661.jpg
[24.67387614]
139
139
./images_reshaped_dim224/10290d40f57b835b5db2d0aa0511056b.jpg
[24.69233487]
139
139
./images_reshaped_dim224/103965484-GettyImages-506104983.1910x1000.jpg
[11.87692229]
156
156
./images_reshaped_dim224/1050f5e214b267acb9d9af60b39fddc8.jpg
[21.57499087]
141
141
./images_reshaped_dim224/105f8100ad9056d210ac8d3f26ed4247.jpg

99
./images_reshaped_dim224/165a39fea84c0cb262fd1ee5feb162d3.jpg
[11.2005785]
156
156
./images_reshaped_dim224/166fc44d3b0a5cc871fa22bc52f1cf3f.jpg
[10.31138494]
157
157
./images_reshaped_dim224/16d68aab69f0d83b60fae04649dda9c8.jpg
[23.46855359]
139
139
./images_reshaped_dim224/16e364601242e39ae5d37c136b8a144d.jpg
[21.68460857]
124
83
./images_reshaped_dim224/16ef594a90645a761a3c71672ba981ec.jpg
[11.6625566]
154
154
./images_reshaped_dim224/16f049354c930a517b432670262e7fef.jpg
[19.74574322]
144
144
./images_reshaped_dim224/171c958db1ebf8178890063cc1d04f4f.jpg
[11.70352838]
154
154
./images_reshaped_dim224/17380-milky-way-galaxy-wallpaper-hd_23086.jpg
[11.422985]
154
154
./images_reshaped_dim224/1745d2ad41b52db3b2d08f8b70b38fe1.jpg
[19.53820471]
143
143
./images_reshaped_dim224/175f29e8ad8c615e73f756b5216e18de.jpg
[22.10330075]
140
140
./images_reshaped_dim224/176ae1eff62305880b099b0ca2e4fb2a.jpg
[13.4593266]
151
151
./images_reshaped_dim224/17771b5193efb7d9905562002a7d94dd.jpg
[18.3940

141
141
./images_reshaped_dim224/1d5fe09d9a50a223882822ad5dee2d6b.jpg
[17.38882375]
146
146
./images_reshaped_dim224/1d6bdbf9033e83950c09bfeebda78002.jpg
[22.02428242]
140
140
./images_reshaped_dim224/1d835bdfb2cceb4ddcf8f23226672a98.jpg
[12.04070809]
154
78
./images_reshaped_dim224/1d8584deb1b7faf86e50eaff9611e1cf.jpg
[22.13737908]
140
140
./images_reshaped_dim224/1d987dfb08dcc71caaf2e1bb9c462365.jpg
[15.20954907]
148
148
./images_reshaped_dim224/1d9b9135a31b69893cbfd43c4b5e9d7e.jpg
[15.0463521]
148
148
./images_reshaped_dim224/1d9da6bb60aa7b1f2b7716972049edda.jpg
[19.82810276]
143
143
./images_reshaped_dim224/1dba89ed73a97521a1798a0d606c40a3.jpg
[14.79546065]
150
150
./images_reshaped_dim224/1dbf14c04731ffa1c3cfff0b347a20f8.jpg
[24.67027179]
139
139
./images_reshaped_dim224/1dbff63e98f02bc0d2b1c326b1771376.jpg
[11.52274136]
154
154
./images_reshaped_dim224/1dcedfd2388aee7e68969c5be0794947.jpg
[14.75136108]
150
150
./images_reshaped_dim224/1dd438a4b4fdaf3dfed7ef5279b192a4.jpg
[16.9683

...

144
./images_reshaped_dim224/P1010594.JPG
[15.96603435]
148
148
./images_reshaped_dim224/P1010629.JPG
[24.44677096]
137
137
./images_reshaped_dim224/P1010634.JPG
[23.90308912]
137
137
./images_reshaped_dim224/P1010635.JPG
[19.53308977]
143
143
./images_reshaped_dim224/P1010642.JPG
[16.5723135]
147
147
./images_reshaped_dim224/P1010644.JPG
[17.9657992]
146
146
./images_reshaped_dim224/P1010658.JPG
[15.19327433]
150
150
./images_reshaped_dim224/P1010660.JPG
[14.64987513]
150
150
./images_reshaped_dim224/P1010723.JPG
[16.04685679]
147
147
./images_reshaped_dim224/P1010777.JPG
[15.87107315]
148
148
./images_reshaped_dim224/P1010801.JPG
[23.05263396]
140
140
./images_reshaped_dim224/P1010831.JPG
[12.32260336]
153
153
./images_reshaped_dim224/P1010875.JPG
[21.38960198]
141
141
./images_reshaped_dim224/P1010881.JPG
[24.87397333]
139
139
./images_reshaped_dim224/P1010893.JPG
[21.08212255]
141
141
./images_reshaped_dim224/P1010930.JPG
[10.26737405]
157
157
./images_reshaped_dim224/P1010979.JPG


148
148
./images_reshaped_dim224/P1040621.JPG
[16.25565297]
147
147
./images_reshaped_dim224/P1040638.JPG
[24.98120997]
137
137
./images_reshaped_dim224/P1040653.JPG
[15.98475151]
147
147
./images_reshaped_dim224/P1040658.JPG
[14.77239616]
150
150
./images_reshaped_dim224/P1040669.JPG
[14.15083228]
150
150
./images_reshaped_dim224/P1040678.JPG
[20.90402196]
141
141
./images_reshaped_dim224/P1040709.JPG
[22.56184195]
140
140
./images_reshaped_dim224/P1040716.JPG
[19.73744547]
144
144
./images_reshaped_dim224/P1040725.JPG
[11.49616739]
154
154
./images_reshaped_dim224/P1040730.JPG
[21.14761576]
143
143
./images_reshaped_dim224/P1040757.JPG
[16.22364691]
147
147
./images_reshaped_dim224/P1040824.JPG
[18.96666518]
143
143
./images_reshaped_dim224/P1040836.JPG
[11.7000349]
154
154
./images_reshaped_dim224/P1040841.JPG
[21.19581534]
143
143
./images_reshaped_dim224/P1040844.JPG
[18.39787784]
146
146
./images_reshaped_dim224/P1040847.JPG
[12.38115525]
153
153
./images_reshaped_dim224/P1040847

154
./images_reshaped_dim224/P1100922.JPG
[23.10326713]
140
140
./images_reshaped_dim224/P1100952.JPG
[15.82195765]
148
148
./images_reshaped_dim224/P1100990.JPG
[24.13851551]
139
139
./images_reshaped_dim224/P1110003.JPG
[11.65944872]
154
154
./images_reshaped_dim224/P1110035.JPG
[13.62777613]
151
151
./images_reshaped_dim224/P1110145.JPG
[24.81735252]
139
139
./images_reshaped_dim224/P1110476.JPG
[12.26233952]
154
154
./images_reshaped_dim224/P1110547.JPG
[21.10471139]
143
143
./images_reshaped_dim224/P1110555.JPG
[21.58181214]
141
141
./images_reshaped_dim224/P1110561.JPG
[16.22162724]
147
147
./images_reshaped_dim224/P1110654.JPG
[13.30803044]
153
153
./images_reshaped_dim224/P1110664.JPG
[17.10335449]
147
78
./images_reshaped_dim224/P1110683.JPG
[19.71244824]
144
143
./images_reshaped_dim224/P1110718.JPG
[10.53976361]
157
157
./images_reshaped_dim224/P1110724.JPG
[24.72257374]
139
83
./images_reshaped_dim224/P1110728.JPG
[13.03093505]
153
153
./images_reshaped_dim224/P1110823.JPG


148
./images_reshaped_dim224/P1200709.JPG
[14.24023046]
151
151
./images_reshaped_dim224/P1200760.JPG
[22.44074143]
140
140
./images_reshaped_dim224/P1200782.JPG
[11.95910184]
156
156
./images_reshaped_dim224/P1200872.JPG
[14.27416691]
151
151
./images_reshaped_dim224/P1200881.JPG
[21.67898112]
141
141
./images_reshaped_dim224/P1200939.JPG
[17.74948673]
144
144
./images_reshaped_dim224/P1210005.JPG
[22.8695069]
140
140
./images_reshaped_dim224/P1210077.JPG
[20.76919219]
141
141
./images_reshaped_dim224/P1210083.JPG
[19.53179169]
143
143
./images_reshaped_dim224/P1210101.JPG
[16.90952946]
147
147
./images_reshaped_dim224/P1210149.JPG
[18.04872164]
144
144
./images_reshaped_dim224/P1210159.JPG
[17.79829657]
144
144
./images_reshaped_dim224/P1210161.JPG
[15.90641087]
148
148
./images_reshaped_dim224/P1210175.JPG
[14.74460336]
150
150
./images_reshaped_dim224/P1210222.JPG
[11.96776243]
156
156
./images_reshaped_dim224/P1210306.JPG
[14.53329418]
150
150
./images_reshaped_dim224/P1210367.JPG

[15.81082051]
148
148
./images_reshaped_dim224/voyage Antoine 2015 1956.JPG
[21.7191296]
141
141
./images_reshaped_dim224/voyage Antoine 2015 1959.JPG
[16.62790974]
147
7
./images_reshaped_dim224/voyage Antoine 2015 199.JPG
[10.26820774]
157
157
./images_reshaped_dim224/voyage Antoine 2015 1997.JPG
[16.64150986]
147
147
./images_reshaped_dim224/voyage Antoine 2015 2029.JPG
[24.34936394]
139
139
./images_reshaped_dim224/voyage Antoine 2015 2033.JPG
[21.81383429]
140
140
./images_reshaped_dim224/voyage Antoine 2015 2102.JPG
[12.659625]
153
153
./images_reshaped_dim224/voyage Antoine 2015 223.JPG
[21.52208751]
141
141
./images_reshaped_dim224/voyage Antoine 2015 2251.JPG
[22.49223753]
140
140
./images_reshaped_dim224/voyage Antoine 2015 2260.JPG
[13.79151729]
150
150
./images_reshaped_dim224/voyage Antoine 2015 2280.JPG
[13.75304828]
150
150
./images_reshaped_dim224/voyage Antoine 2015 2345.JPG
[11.46709026]
154
154
./images_reshaped_dim224/voyage Antoine 2015 2379.JPG
[10.85448697]
157
1